In [51]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
#make order while extracting importent incredient
from collections import OrderedDict

# Sample data with categories for each ingredient
data = {
    'Recipe': [
        'Rotti', 'Pasta', 'Pittu', 'Thosa', 'Idli', 'Vadai', 'Ulundhu Vadai', 'Uppuma', 'Hopper (Aappa)',
        'Noodles', 'Bun', 'Pizza', 'Patties', 'Pastry', 'Cake', 'Pancake', 'Briyani', 'Fried Rice',
        'Milk Rice (Kiri Badh)', 'Pongal', 'Doughnut', 'Kanji', 'Sandwich', 'String Hopper', 'Rice', 'Pulses'
    ],
    'Ingredients': [
        'Wheat Flour, Coconut, Butter',
        'Pasta, Carrot, Chicken, Egg, Soy Sauce, Tomato Sauce',
        'Rice Flour, Coconut',
        'Wheat Flour, Ulundhu, Butter, Baking Soda',
        'Rice, Ulundhu, Baking Soda',
        'Bengal Gram, Chilli, Onion',
        'Ulundhu, Rice Flour, Onion, Chilli',
        'Rava, Carrot, Onion',
        'Rice Flour, Coconut, Egg, Baking Soda',
        'Spaghetti, Egg, Soy Sauce, Carrot, Chicken, Tomato Sauce',
        'Wheat Flour, Egg, Butter, Yeast',
        'Wheat Flour, Egg, Butter, Cheese, Yeast, Tomato Sauce',
        'Wheat Flour, Egg, Butter, Potato, Onion',
        'Wheat Flour, Butter, Baking Powder',
        'Wheat Flour, Butter, Egg, Baking Powder, Sugar',
        'Wheat Flour, Milk, Egg, Baking Powder',
        'Rice, Chicken, Onion, Potato',
        'Rice, Carrot, Soy Sauce, Onion, Egg, Chicken',
        'Rice, Coconut, Sugar',
        'Rice, Coconut, Jaggery, Green Gram',
        'Wheat Flour, Milk, Butter, Yeast, Sugar',
        'Rice, Coconut, Chicken, Carrot',
        'Bread, Cheese, Chicken, Tomato Sauce',
        'Rice Flour',
        'Rice',
        'Chickpeas, Green Gram, Kidney Beans'
    ]
}

df = pd.DataFrame(data)

# Preprocess the data
df['Ingredients'] = df['Ingredients'].str.replace(',', ' ')

# Create a CountVectorizer to convert text into a matrix of token counts
vectorizer = CountVectorizer()
ingredient_matrix = vectorizer.fit_transform(df['Ingredients'])

# Compute cosine similarity
cosine_sim = cosine_similarity(ingredient_matrix, ingredient_matrix)

# Function to recommend recipes based on selected ingredients
def get_recipe_recommendations(selected_ingredients, cosine_sim, df):
    # Convert both selected ingredients and ingredients in DataFrame to lowercase
    selected_ingredients_lower = [ing.lower() for ing in selected_ingredients]
    print("\nselected incredients are :", selected_ingredients_lower)
    df['Ingredients_lower'] = df['Ingredients'].str.lower()

    # Extracting unique first ingredients in order
    main_ingredient = list(OrderedDict.fromkeys(ingredient.split(', ')[0] for ingredient in data['Ingredients']))
    print("\nmain incredients are :", main_ingredient)

    if not any(ingredient.lower() in selected_ingredients_lower for ingredient in main_ingredient):
    # if not any(main_ingredient.isin(selected_ingredients_lower)):
        print(f"\nThe main ingredient to make these recipes is not in the selected ingredients.")
        return [], []

    # Combine selected ingredients into a query
    query = ' '.join(selected_ingredients_lower)

    # Transform the query using the CountVectorizer
    query_vectorized = vectorizer.transform([query])

    # Compute cosine similarity between the query and recipes
    cosine_scores = cosine_similarity(query_vectorized, ingredient_matrix)

    # Get indices of recipes sorted by similarity
    recipe_indices = cosine_scores.argsort()[0][::-1]

    # Initialize a dictionary to store recipe scores based on matching ingredients
    recipe_scores = {}
    for index in recipe_indices:
        recipe = df['Recipe'].iloc[index]
        ingredients = df['Ingredients_lower'].iloc[index].split()
        matched_count = sum(1 for ing in selected_ingredients_lower if ing in ingredients)
        recipe_scores[recipe] = matched_count

    # Sort recipes based on the number of matching ingredients
    sorted_recipes = sorted(recipe_scores.items(), key=lambda x: x[1], reverse=True)

    # Return the top 5 recipes and the single best recipe
    top_5_recipes = [recipe[0] for recipe in sorted_recipes[:5]]
    single_best_recipe = [sorted_recipes[0][0]] if sorted_recipes else []

    return top_5_recipes, single_best_recipe

# Function to get user input and display recommendations
def get_user_input_and_display_recommendations():
    print("Available recipes:")
    print(df['Recipe'].values)

    # Sample ingredient categories
    categories = {
        'Proteins': ['Chicken', 'Egg'],
        'Vegetables': ['Carrot', 'Onion', 'Potato', 'Chilli', 'Coconut'],
        'Main': ['Wheat Flour', 'Pasta', 'Rice Flour','Rice', 'Bengal Gram','Ulundhu','Rava','Spaghetti', 'Bread','Chickpeas' ],
        'Lentils': ['Green Gram', 'Kidney Beans'],
        'Dairy': ['Milk', 'Butter', 'Cheese'],
        'Leavening Agents': ['Baking Powder', 'Baking Soda', 'Yeast'],
        'Sauces': ['Soy Sauce', 'Tomato Sauce'],
        'Sweeteners': ['Sugar', 'Jaggery']
    }

    # Collect user input based on categories
    selected_ingredients = []
    for category, ingredients in categories.items():
        print(f"\n{category}: {', '.join(ingredients)}")
        user_choice = input(f"Select {category.lower()} ingredients (comma-separated): ")
        selected_ingredients.extend([ing.strip() for ing in user_choice.split(',') if ing.strip() in ingredients])

    print("\nSelected incredients are")
    print(selected_ingredients)
    print("Calling get_recipie function\n")
    if selected_ingredients:
        top_5_recipes, best_single_recipe = get_recipe_recommendations(selected_ingredients, cosine_sim, df)

        print('\nTop 5 Recommended Recipes:')
        for recipe in top_5_recipes:
            print('- ' + recipe)

        print('\nSingle Best Recommended Recipe:')
        if best_single_recipe:
            print('- ' + best_single_recipe[0])
        else:
            print('No single best recipe found.')

        if not top_5_recipes and not best_single_recipe:
            print('\nNo recipes found based on the selected ingredients.')
            print('Would you like to order food online?')
            order_choice = input('Enter "Yes" to order food online, or any key to exit: ')

# Example usage: Recommend recipes based on user input
get_user_input_and_display_recommendations()

Available recipes:
['Rotti' 'Pasta' 'Pittu' 'Thosa' 'Idli' 'Vadai' 'Ulundhu Vadai' 'Uppuma'
 'Hopper (Aappa)' 'Noodles' 'Bun' 'Pizza' 'Patties' 'Pastry' 'Cake'
 'Pancake' 'Briyani' 'Fried Rice' 'Milk Rice (Kiri Badh)' 'Pongal'
 'Doughnut' 'Kanji' 'Sandwich' 'String Hopper' 'Rice' 'Pulses']

Proteins: Chicken, Egg
Select proteins ingredients (comma-separated): Egg

Vegetables: Carrot, Onion, Potato, Chilli, Coconut
Select vegetables ingredients (comma-separated): Onion

Main: Wheat Flour, Pasta, Rice Flour, Rice, Bengal Gram, Ulundhu, Rava, Spaghetti, Bread, Chickpeas
Select main ingredients (comma-separated): Pasta

Lentils: Green Gram, Kidney Beans
Select lentils ingredients (comma-separated): Kidney Beans

Dairy: Milk, Butter, Cheese
Select dairy ingredients (comma-separated): Butter

Leavening Agents: Baking Powder, Baking Soda, Yeast
Select leavening agents ingredients (comma-separated): Baking Soda

Sauces: Soy Sauce, Tomato Sauce
Select sauces ingredients (comma-separated): Tomat

**Final**

In [52]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from collections import OrderedDict
import ipywidgets as widgets
from IPython.display import display

# Sample data with categories for each ingredient
data = {
    'Recipe': [
        'Rotti', 'Pasta', 'Pittu', 'Thosa', 'Idli', 'Vadai', 'Ulundhu Vadai', 'Uppuma', 'Hopper (Aappa)',
        'Noodles', 'Bun', 'Pizza', 'Patties', 'Pastry', 'Cake', 'Pancake', 'Briyani', 'Fried Rice',
        'Milk Rice (Kiri Badh)', 'Pongal', 'Doughnut', 'Kanji', 'Sandwich', 'String Hopper', 'Rice', 'Pulses'
    ],
    'Ingredients': [
        'Wheat Flour, Coconut, Butter',
        'Pasta, Carrot, Chicken, Egg, Soy Sauce, Tomato Sauce',
        'Rice Flour, Coconut',
        'Wheat Flour, Ulundhu, Butter, Baking Soda',
        'Rice, Ulundhu, Baking Soda',
        'Bengal Gram, Chilli, Onion',
        'Ulundhu, Rice Flour, Onion, Chilli',
        'Rava, Carrot, Onion',
        'Rice Flour, Coconut, Egg, Baking Soda',
        'Spaghetti, Egg, Soy Sauce, Carrot, Chicken, Tomato Sauce',
        'Wheat Flour, Egg, Butter, Yeast',
        'Wheat Flour, Egg, Butter, Cheese, Yeast, Tomato Sauce',
        'Wheat Flour, Egg, Butter, Potato, Onion',
        'Wheat Flour, Butter, Baking Powder',
        'Wheat Flour, Butter, Egg, Baking Powder, Sugar',
        'Wheat Flour, Milk, Egg, Baking Powder',
        'Rice, Chicken, Onion, Potato',
        'Rice, Carrot, Soy Sauce, Onion, Egg, Chicken',
        'Rice, Coconut, Sugar',
        'Rice, Coconut, Jaggery, Green Gram',
        'Wheat Flour, Milk, Butter, Yeast, Sugar',
        'Rice, Coconut, Chicken, Carrot',
        'Bread, Cheese, Chicken, Tomato Sauce',
        'Rice Flour',
        'Rice',
        'Chickpeas, Green Gram, Kidney Beans'
    ]
}

df = pd.DataFrame(data)

# Preprocess the data
df['Ingredients'] = df['Ingredients'].str.replace(',', ' ')

# Create a CountVectorizer to convert text into a matrix of token counts
vectorizer = CountVectorizer()
ingredient_matrix = vectorizer.fit_transform(df['Ingredients'])

# Compute cosine similarity
cosine_sim = cosine_similarity(ingredient_matrix, ingredient_matrix)

# Function to recommend recipes based on selected ingredients
def get_recipe_recommendations(selected_ingredients, cosine_sim, df):
    # Convert both selected ingredients and ingredients in DataFrame to lowercase
    selected_ingredients_lower = [ing.lower() for ing in selected_ingredients]
    print("\nselected ingredients are :", selected_ingredients_lower)
    df['Ingredients_lower'] = df['Ingredients'].str.lower()

    # Extracting unique first ingredients in order
    main_ingredient = list(OrderedDict.fromkeys(ingredient.split(', ')[0] for ingredient in data['Ingredients']))
    print("\nmain ingredients are :", main_ingredient)

    if not any(ingredient.lower() in selected_ingredients_lower for ingredient in main_ingredient):
        print(f"\nThe main ingredient to make these recipes is not in the selected ingredients.")
        return [], []

    # Combine selected ingredients into a query
    query = ' '.join(selected_ingredients_lower)

    # Transform the query using the CountVectorizer
    query_vectorized = vectorizer.transform([query])

    # Compute cosine similarity between the query and recipes
    cosine_scores = cosine_similarity(query_vectorized, ingredient_matrix)

    # Get indices of recipes sorted by similarity
    recipe_indices = cosine_scores.argsort()[0][::-1]

    # Initialize a dictionary to store recipe scores based on matching ingredients
    recipe_scores = {}
    for index in recipe_indices:
        recipe = df['Recipe'].iloc[index]
        ingredients = df['Ingredients_lower'].iloc[index].split()
        matched_count = sum(1 for ing in selected_ingredients_lower if ing in ingredients)
        recipe_scores[recipe] = matched_count

    # Sort recipes based on the number of matching ingredients
    sorted_recipes = sorted(recipe_scores.items(), key=lambda x: x[1], reverse=True)

    # Return the top 5 recipes and the single best recipe
    top_5_recipes = [recipe[0] for recipe in sorted_recipes[:5]]
    single_best_recipe = [sorted_recipes[0][0]] if sorted_recipes else []

    return top_5_recipes, single_best_recipe

# ... (previous code)

# Function to get user input and display recommendations
# ... (previous code)

def get_user_input_and_display_recommendations():
    print("Available recipes:")
    print(df['Recipe'].values)

    # Sample ingredient categories
    categories = {
        'Proteins': ['Chicken', 'Egg'],
        'Vegetables': ['Carrot', 'Onion', 'Potato', 'Chilli', 'Coconut'],
        'Main': ['Wheat Flour', 'Pasta', 'Rice Flour', 'Rice', 'Bengal Gram', 'Ulundhu', 'Rava', 'Spaghetti',
                 'Bread', 'Chickpeas'],
        'Lentils': ['Green Gram', 'Kidney Beans'],
        'Dairy': ['Milk', 'Butter', 'Cheese'],
        'Leavening Agents': ['Baking Powder', 'Baking Soda', 'Yeast'],
        'Sauces': ['Soy Sauce', 'Tomato Sauce'],
        'Sweeteners': ['Sugar', 'Jaggery']
    }

    # Collect user input based on categories
    global selected_ingredients
    selected_ingredients = []

    def on_dropdown_change(change, category):
        selected_ingredient = change["new"]
        print(f"\nSelected ingredient for {category} is: {selected_ingredient}")
        selected_ingredients.append(selected_ingredient)

    for category, ingredients in categories.items():
        print(f"\n{category}: {', '.join(ingredients)}")
        dropdown = widgets.Dropdown(options=ingredients, description=f"Select {category.lower()} ingredients:")

        # Attach the event handler to the dropdown widget
        dropdown.observe(lambda change, category=category: on_dropdown_change(change, category), names='value')

        display(dropdown)

    # Use a button to signal when the user has finished selecting
    button = widgets.Button(description="Finished Selection")
    display(button)

    def on_button_click(b):
        global selected_ingredients  # Declare selected_ingredients as global
        print("\nFinal selected ingredients are:")
        print(selected_ingredients)

        if selected_ingredients:
            # Remove duplicates
            selected_ingredients = list(set(selected_ingredients))

            top_5_recipes, best_single_recipe = get_recipe_recommendations(selected_ingredients, cosine_sim, df)

            print('\nTop 5 Recommended Recipes:')
            for recipe in top_5_recipes:
                print('- ' + recipe)

            print('\nSingle Best Recommended Recipe:')
            if best_single_recipe:
                print('- ' + best_single_recipe[0])
            else:
                print('No single best recipe found.')

            if not top_5_recipes and not best_single_recipe:
                print('\nNo recipes found based on the selected ingredients.')
                print('Would you like to order food online?')

                order_choice = input('Enter "Yes" to order food online, or any key to exit: ')
                if order_choice.lower() == 'yes':
                    print('Redirecting to online food ordering platform...')
                    # Add your code to redirect to an online food ordering platform
                else:
                    print('Exiting. Have a good day!')

    # Attach the event handler to the button widget
    button.on_click(on_button_click)

# Example usage: Recommend recipes based on user input
get_user_input_and_display_recommendations()




Available recipes:
['Rotti' 'Pasta' 'Pittu' 'Thosa' 'Idli' 'Vadai' 'Ulundhu Vadai' 'Uppuma'
 'Hopper (Aappa)' 'Noodles' 'Bun' 'Pizza' 'Patties' 'Pastry' 'Cake'
 'Pancake' 'Briyani' 'Fried Rice' 'Milk Rice (Kiri Badh)' 'Pongal'
 'Doughnut' 'Kanji' 'Sandwich' 'String Hopper' 'Rice' 'Pulses']

Proteins: Chicken, Egg


Dropdown(description='Select proteins ingredients:', options=('Chicken', 'Egg'), value='Chicken')


Vegetables: Carrot, Onion, Potato, Chilli, Coconut


Dropdown(description='Select vegetables ingredients:', options=('Carrot', 'Onion', 'Potato', 'Chilli', 'Coconu…


Main: Wheat Flour, Pasta, Rice Flour, Rice, Bengal Gram, Ulundhu, Rava, Spaghetti, Bread, Chickpeas


...

Dropdown(description='Select sauces ingredients:', options=('Soy Sauce', 'Tomato Sauce'), value='Soy Sauce')


Sweeteners: Sugar, Jaggery


Dropdown(description='Select sweeteners ingredients:', options=('Sugar', 'Jaggery'), value='Sugar')

Button(description='Finished Selection', style=ButtonStyle())


Selected ingredient for Proteins is: Egg

Selected ingredient for Vegetables is: Onion

Selected ingredient for Main is: Pasta

Selected ingredient for Lentils is: Kidney Beans

Selected ingredient for Dairy is: Butter

Selected ingredient for Leavening Agents is: Baking Soda

Selected ingredient for Sauces is: Tomato Sauce

Selected ingredient for Sweeteners is: Jaggery

Final selected ingredients are:
['Egg', 'Onion', 'Pasta', 'Kidney Beans', 'Butter', 'Baking Soda', 'Tomato Sauce', 'Jaggery']

selected ingredients are : ['egg', 'jaggery', 'kidney beans', 'baking soda', 'pasta', 'butter', 'onion', 'tomato sauce']

main ingredients are : ['Wheat Flour', 'Pasta', 'Rice Flour', 'Rice', 'Bengal Gram', 'Ulundhu', 'Rava', 'Spaghetti', 'Bread', 'Chickpeas']

Top 5 Recommended Recipes:
- Patties
- Pasta
- Pizza
- Cake
- Fried Rice

Single Best Recommended Recipe:
- Patties
